#Approccio Zero-shot

## Installazione dei pacchetti

In [ ]:
# Disistalliamo le versioni incompatibili
!pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126


In [ ]:
# Aggiorna PyTorch e Torchaudio
!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu126

# Librerie fondamentali
!pip install huggingface_hub unidecode phonemizer einops torchlibrosa transformers timm librosa ftfy --quiet

# Modelli della pipeline
!pip install demucs asteroid --quiet

# Clona repo e download AudioSR
%cd /content
!git clone https://github.com/haoheliu/versatile_audio_super_resolution.git
%cd versatile_audio_super_resolution

# Rimozione della riga che blocca torch==2.0.1
!sed -i '/torch==2.0.1+/d' requirements.txt

# Download di AudioSR ignorando i vincoli di versione
!pip install -r requirements.txt --no-deps --quiet
!pip install -e . --quiet


#Scarica script di inferenza Colab-friendly
!wget https://raw.githubusercontent.com/jarredou/AudioSR-Colab-Fork/main/inference.py -q

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.8/821.8 MB 762.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━

## Import

In [ ]:
import gc
import os
import shutil
import glob
import random
import numpy as np
import torch
import torchaudio
import librosa
import soundfile as sf
import tempfile, subprocess
import pandas as pd
import contextlib
import io
import IPython.display as ipd
from pathlib import Path
from torchmetrics.audio import SignalNoiseRatio, ScaleInvariantSignalDistortionRatio
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm


# Modelli principali
import demucs
import asteroid

# AudioSR
from audiosr import build_model, super_resolution

print("Tutti i pacchetti importati correttamente")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Tutti i pacchetti importati correttamente


/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


##Dataset

###Download

In [ ]:
# Torna nella root principale di Colab
%cd /content

# Scarica FMA Small
!wget https://os.unil.cloud.switch.ch/fma/fma_small.zip
!unzip fma_small.zip

Output streaming troncato alle ultime 5000 righe.
 bunzipping: fma_small/068/068600.mp3  
 bunzipping: fma_small/068/068601.mp3  
 bunzipping: fma_small/068/068680.mp3  
 bunzipping: fma_small/068/068682.mp3  
 bunzipping: fma_small/068/068683.mp3  
 bunzipping: fma_small/068/068820.mp3  
 bunzipping: fma_small/068/068821.mp3  
 bunzipping: fma_small/068/068837.mp3  
 bunzipping: fma_small/068/068838.mp3  
 bunzipping: fma_small/068/068839.mp3  
 bunzipping: fma_small/068/068840.mp3  
 bunzipping: fma_small/068/068841.mp3  
 bunzipping: fma_small/068/068842.mp3  
 bunzipping: fma_small/068/068843.mp3  
 bunzipping: fma_small/068/068844.mp3  
 bunzipping: fma_small/068/068851.mp3  
 bunzipping: fma_small/068/068852.mp3  
 bunzipping: fma_small/068/068853.mp3  
 bunzipping: fma_small/068/068854.mp3  
 bunzipping: fma_small/068/068860.mp3  
 bunzipping: fma_small/068/068861.mp3  
 bunzipping: fma_small/068/068862.mp3  
 bunzipping: fma_small/068/068869.mp3  
 bunzipping: fma_small/068/068

###Campionamento

In [ ]:
# Percorsi
src_dir = Path("fma_small") # cartella appena estratta
dst_dir = Path("fma_clips") # dove le clip vengono salvate
dst_dir.mkdir(exist_ok=True)

# Lista di tutti i file .mp3
all_tracks = list(src_dir.glob("**/*.mp3"))
print(f"Trovate {len(all_tracks)} tracce totali.")

# Campionamento di 100 tracce casuali
n_clips = 100
sampled = random.sample(all_tracks, n_clips)

# Estrazione di una clip da 5 secondi per ogni traccia
sr_target = 32000
clip_len = 5 * sr_target

for i, track in enumerate(sampled):
    try:
        y, sr = librosa.load(track, sr=sr_target)
        if len(y) > clip_len:
            start = random.randint(0, len(y) - clip_len)
            clip = y[start:start + clip_len]
        else:
            # se la traccia è più corta, la usiamo tutta
            clip = y
        sf.write(dst_dir / f"clip_{i:03d}.wav", clip, sr_target)
    except Exception as e:
        print(f"Errore con {track}: {e}")

print(f"{len(sampled)} clip da 5 s salvate in {dst_dir.resolve()}")

Trovate 8000 tracce totali.
100 clip da 5 s salvate in /content/fma_clips


In [ ]:
# Rimuove i file .zip nella root di /content
for f in glob.glob("/content/*.zip"):
    try:
        os.remove(f)
        print(f" Eliminato file ZIP: {f}")
    except Exception as e:
        print(f" Errore eliminando {f}: {e}")

# Rimuove la cartella estratta (se presente)
for folder in glob.glob("/content/fma_*"):
    if "fma_clips" not in folder and os.path.isdir(folder):
        try:
            shutil.rmtree(folder)
            print(f" Rimossa cartella: {folder}")
        except Exception as e:
            print(f" Errore rimuovendo {folder}: {e}")

print(" Pulizia completata. Solo fma_clips mantenuta.")


 Eliminato file ZIP: /content/fma_small.zip
 Rimossa cartella: /content/fma_small
 Pulizia completata. Solo fma_clips mantenuta.


###Classe Dataset

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, clip_dir, sr=32000, degradation_types=['quantize', 'lowpass', 'reverb', 'clip']):
        self.files = list(Path(clip_dir).glob("*.wav"))
        self.sr = sr
        self.degradation_types = degradation_types

    def __len__(self):
        return len(self.files)

    def degradation(self, audio, sr=32000, degradation_types=['quantize', 'lowpass', 'reverb', 'clip']):

        x = audio.copy()

        # Quantizzazione bit-depth + dithering
        if 'quantize' in degradation_types:
            bits = np.random.choice([8, 10, 12])
            q = 2 ** bits
            dither = np.random.uniform(-0.5/q, 0.5/q, size=x.shape)
            x = np.round(x * q) / q + dither
            x = np.clip(x, -1.0, 1.0)

        # Low-pass realistico
        if 'lowpass' in degradation_types:
            cutoff = float(np.random.uniform(3500, 8000))
            xt = torch.from_numpy(x).unsqueeze(0)
            xt = torchaudio.functional.lowpass_biquad(xt, sr, cutoff)
            x = xt.squeeze(0).numpy()


        # Riverbero artificiale (coda esponenziale)
        if 'reverb' in degradation_types:
            reverb_ms = np.random.uniform(50, 200)  # lunghezza in ms
            reverb_samples = int(self.sr * reverb_ms / 1000)
            ir = np.exp(-np.linspace(0, 5, reverb_samples))  # impulso decrescente
            x = np.convolve(x, ir, mode='full')[:len(x)]
            x = x / (np.max(np.abs(x)) + 1e-9)

        # Clipping leggero
        if 'clip' in degradation_types:
            clip_thr = float(np.random.uniform(0.8, 0.95))
            x = np.clip(x, -clip_thr, clip_thr)

        return x

    def __getitem__(self, idx):
        path = self.files[idx]
        audio, _ = librosa.load(path, sr=self.sr, mono=True)
        degraded = self.degradation(audio, sr=self.sr, degradation_types=self.degradation_types)
        return torch.tensor(degraded).float(), torch.tensor(audio).float(), str(path)


In [ ]:
dataset = AudioDataset("/content/fma_clips")
loader = DataLoader(dataset, batch_size=1, shuffle=True)

## Istanziamo i modelli

In [ ]:
from demucs.pretrained import get_model as demucs_get_model
from asteroid.models import BaseModel

print("Caricamento modelli...")
demucs_model = demucs_get_model('htdemucs').to(device).eval()

model_id = "JorisCos/ConvTasNet_Libri1Mix_enhsingle_16k"
asteroid_model = BaseModel.from_pretrained(model_id).to(device).to(device).eval()  # denoising vocale

Caricamento modelli...
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th


100%|██████████| 80.2M/80.2M [00:00<00:00, 155MB/s]


pytorch_model.bin:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

In [ ]:
print(" Pulizia memoria precedente...")
# Libera eventuali vecchie istanze del modello
try:
    del audiosr_model
except NameError:
    pass

gc.collect()
torch.cuda.empty_cache()

# Pulire cache di torch/huggingface se piena
for cache_dir in ["/root/.cache/torch/hub", "/root/.cache/huggingface"]:
    if os.path.exists(cache_dir):
        try:
            shutil.rmtree(cache_dir)
            print(f" Cache rimossa: {cache_dir}")
        except Exception as e:
            print(f" Errore rimuovendo {cache_dir}: {e}")


# Caricamento del modello AudioSR
try:
    audiosr_model = build_model("basic", device=device)

    audiosr_model.to(device)
    print(f"AudioSR caricato con successo su {device} ({type(audiosr_model).__name__})")

except Exception as e:
    print(f"Errore durante il caricamento di AudioSR: {e}")

 Pulizia memoria precedente...
 Cache rimossa: /root/.cache/torch/hub
 Cache rimossa: /root/.cache/huggingface
Loading AudioSR: basic
Loading model on cuda


pytorch_model.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.12/dist-packages/torchaudio/transforms/_transforms.py:581: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


DiffusionWrapper has 258.20 M params.


/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


AudioSR caricato con successo su cuda (LatentDiffusion)


## Pipeline

In [ ]:
%cd /content/versatile_audio_super_resolution
from versatile_audio_super_resolution.audiosr.pipeline import make_batch_for_super_resolution

def super_resolve_tensor(model, audio_tensor, sr=32000):
    """
    Esegue AudioSR direttamente su un tensore PyTorch e restituisce
    un waveform 1D normalizzato (senza barre o stampe).
    """
    model.training = False  # evita ricorsione da .eval()

    # Pre-elaborazione
    if audio_tensor.dim() == 2:
        audio_tensor = audio_tensor.mean(0)  # stereo → mono

    waveform_np = audio_tensor.detach().cpu().numpy().astype("float32")

    # Crea batch per AudioSR
    batch, duration = make_batch_for_super_resolution(
        input_file=None,
        waveform=waveform_np
    )

    # Inferenza silenziata
    import contextlib, io
    with torch.no_grad(), contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
        output = model.generate_batch(
            batch,
            unconditional_guidance_scale=3.0,
            ddim_steps=50,
        )

    # Normalizza e rimuove dimensioni extra
    out = torch.tensor(output).float().squeeze()

    return out.cpu()

/content/versatile_audio_super_resolution


In [ ]:
def normalize_audio(x):
    return x / (x.abs().max() + 1e-9)

In [ ]:
from demucs.apply import apply_model

# Metriche
snr_metric = SignalNoiseRatio().to("cpu")
sisdr_metric = ScaleInvariantSignalDistortionRatio().to("cpu")

results = []

# Barra notebook-safe
with tqdm(total=len(loader), desc="Pipeline per-stem (Demucs + Asteroid + AudioSR)", leave=True) as pbar:
    for degraded, clean, path in loader:

        degraded = degraded.squeeze(0).to(device)
        clean = clean.squeeze(0).cpu().numpy()

        with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16):
            # Prepara input stereo per Demucs
            x = degraded.unsqueeze(0).unsqueeze(0)
            if x.shape[1] == 1:
                x = x.repeat(1, 2, 1)

            # Separazione Demucs
            demucs_out = apply_model(demucs_model, x, split=True, overlap=0.25)[0]
            demucs_out = demucs_out.mean(1)

            vocals = demucs_out[0].unsqueeze(0)
            others = demucs_out[1:].sum(0, keepdim=True)

            # Asteroid solo sulla voce
            vocals_enh = asteroid_model.separate(vocals)[0]
            mix_enh = vocals_enh + others

        # AudioSR
        sr_audio = super_resolve_tensor(audiosr_model, mix_enh.to("cpu"))


        # Metriche
        enhanced_t = torch.as_tensor(sr_audio).float().squeeze()
        clean_t = torch.tensor(clean).float().squeeze()
        degraded_t = degraded.cpu().float().squeeze()

        # Uniforma lunghezze
        min_len = min(len(enhanced_t), len(clean_t))
        enhanced_t = enhanced_t[:min_len]
        clean_t = clean_t[:min_len]
        degraded_t = degraded_t[:min_len]

        # Correzione polarità
        if torch.mean(enhanced_t * clean_t) < 0:
            enhanced_t = -enhanced_t

        # Normalizzazione
        enhanced_t = normalize_audio(enhanced_t)
        clean_t = normalize_audio(clean_t)
        degraded_t = normalize_audio(degraded_t)

        # Ascolto qualitativo solo per la prima clip
        if len(results) == 0:
            sf.write("clean_first.wav", clean_t.numpy(), 32000)
            sf.write("degraded_first.wav", degraded_t.numpy(), 32000)
            sf.write("enhanced_first.wav", enhanced_t.numpy(), 32000)

            print(f"\nConfronto qualitativo sulla clip: {path[0].split('/')[-1]}")
            print("Clean:")
            display(ipd.Audio("clean_first.wav"))
            print("Degraded:")
            display(ipd.Audio("degraded_first.wav"))
            print("Enhanced (Demucs + Asteroid + AudioSR):")
            display(ipd.Audio("enhanced_first.wav"))

        # Aggiungi batch dim
        enhanced_t = enhanced_t.unsqueeze(0)
        clean_t = clean_t.unsqueeze(0)
        degraded_t = degraded_t.unsqueeze(0)

        # Calcolo metriche
        snr_deg = snr_metric(degraded_t, clean_t).item()
        snr_enh = snr_metric(enhanced_t, clean_t).item()
        sisdr_deg = sisdr_metric(degraded_t, clean_t).item()
        sisdr_enh = sisdr_metric(enhanced_t, clean_t).item()

        results.append({
            "clip": path[0].split("/")[-1],
            "snr_deg": snr_deg,
            "snr_enh": snr_enh,
            "delta_snr": snr_enh - snr_deg,
            "sisdr_deg": sisdr_deg,
            "sisdr_enh": sisdr_enh,
            "delta_sisdr": sisdr_enh - sisdr_deg
        })

        torch.cuda.empty_cache()
        pbar.update(1)


# RISULTATI
df = pd.DataFrame(results)

print("Pipeline completata su", len(df), "clip\n")
display(df.head())

print("\nMedie globali:")
print(df[["snr_deg", "snr_enh", "delta_snr", "sisdr_deg", "sisdr_enh", "delta_sisdr"]].mean())


Pipeline per-stem (Demucs + Asteroid + AudioSR):   0%|          | 0/100 [00:00<?, ?it/s]


Confronto qualitativo sulla clip: clip_061.wav
Clean:


Degraded:


Enhanced (Demucs + Asteroid + AudioSR):


Pipeline completata su 100 clip



,clip,snr_deg,snr_enh,delta_snr,sisdr_deg,sisdr_enh,delta_sisdr
0,clip_061.wav,-3.640945,-3.077617,0.563328,-15.562123,-15.703850,-0.141726
1,clip_028.wav,-3.288093,-2.347665,0.940428,-24.645975,-24.624260,0.021715
2,clip_074.wav,-4.249083,-0.461362,3.787721,-27.175228,-28.133015,-0.957787
3,clip_060.wav,-2.701685,-2.576375,0.125310,-40.037937,-37.965794,2.072144
4,clip_073.wav,-5.204064,-2.503020,2.701045,-10.958944,-15.598568,-4.639624



Medie globali:
snr_deg        -3.603271
snr_enh        -2.224055
delta_snr       1.379216
sisdr_deg     -29.011523
sisdr_enh     -32.689040
delta_sisdr    -3.677518
dtype: float64
